In [ ]:
# To Do:
# Clean up Offensive Stats loop

In [139]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import datetime
import pickle
pd.set_option('display.max_rows', None)  # Maximize rows visible

### Web Scraping Function

In [57]:
def scrape_table(url):
    response=requests.get(url)
    if response.status_code != 200:
        print('Status code error.')
    soup=BeautifulSoup(response.text, 'html.parser')
    rows=soup.find_all('tr')
    data=[]
    for row in rows:
        cols=row.find_all(['th','td'])
        row_data = []
        # Account for columns that may contain an images as values
        for col in cols:
            img = col.find('img')  # Check for an image inside each cell
            # Extract text only whether there is an image in the cell or not
            if img:
                row_data.append(col.text.strip()) 
            else:
                row_data.append(col.text.strip())  
        data.append(row_data)
    global df  # Initialize global variable for subsequent table renaming  
    df = pd.DataFrame(data)
    return df

### Constants

In [84]:
current_year = datetime.datetime.now().year
years = [str(year) for year in range(1970, current_year)]

### Offensive Stats

In [141]:
offensive_stats = ['passing', 'rushing', 'receiving']
offensive_stats_dict = {}
# This loop scrapes 162 tables from the web and is thus computationally expensive and time-consuming. Therefore, this loop is only run once, then each dataframe is stored in a dictionary locally as a pickle file for further data anaylsis.
# Remove "if False" clause to run the loop for the first time
if False:
    for stat in offensive_stats:   
        # Scrape all offensive stats for all years
        for year in years:
            url=f'https://www.nfl.com/stats/player-stats/category/{stat}/{year}/reg/all/{stat}yards/desc'
            scrape_table(url)
            # Apply transformations
            df.columns = df.iloc[0]
            df = df.iloc[1:]
            if stat == 'passing':
                df=df.rename(columns={'Pass Yds':'Total Yards','Yds/Att':'Yards/Attempt','Att':'Pass Attempts','Cmp':'Completions',
                                      'Cmp %':'Completion %','TD':'Touchdowns','INT':'Interceptions','Rate':'Passer Rating',
                                      '20+':'20+ Yard Completions','40+':'40+ Yard Completions','Lng':'Longest Completion','Sck':'Sacks Taken'})
                df=df[['Player','Total Yards','Pass Attempts','Yards/Attempt','Completions','Completion %','Touchdowns','Interceptions',
                'Passer Rating','20+ Yard Completions','40+ Yard Completions','Longest Completion','Sacks Taken']]
            elif stat == 'rushing':
                df=df.rename(columns={'Rush Yds':'Total Yards','Att':'Rush Attempts','TD':'Touchdowns','20+':'20+ Yard Runs',
                                      '40+':'40+ Yard Runs','Lng':'Longest Run','Rush FUM':'Fumbles'})
                df=df[['Player','Total Yards','Rush Attempts','Touchdowns','20+ Yard Runs','40+ Yard Runs','Longest Run','Fumbles']]
            elif stat == 'receiving':
                df=df.rename(columns={'Rec':'Receptions','Yds':'Total Yards','TD':'Touchdowns','20+':'20+ Yard Receptions',
                                      '40+':'40+ Yard Receptions','LNG':'Longest Reception','Rec FUM':'Fumbles','Rec YAC/R':'Yards After Catch',
                                      'Tgts':'Targets'})
                df=df[['Player','Receptions','Total Yards','Touchdowns','Targets','Yards After Catch','20+ Yard Receptions','40+ Yard Receptions',
                'Longest Reception','Fumbles']]
            # Initialize variable for each unique stat and year dataframe
            globals()[f'{stat}_stats_{year}'] = df
            # Store each dataframe in a dictionary with same variable naming convention as a string
            offensive_stats_dict[f'{stat}_stats_{year}'] = globals()[f'{stat}_stats_{year}']
            # Store dictionary as a pkl file 
            with open('offensive_stats_dict.pkl', 'wb') as b:
                pickle.dump(offensive_stats_dict, b)

In [ ]:
# Identical renames: Yds, TD, 20+, 40+, Lng (upper)
# Similar renames: Att/Tgts

In [147]:
# Load dictionary from pkl file storage. 
# This will allow you to query all previously scraped tables for data analysis without having to run the loop again.
with open('offensive_stats_dict.pkl', 'rb') as b:
    offensive_stats_dict = pickle.load(b)
offensive_stats_dict['rushing_stats_2018']

,Player,Total Yards,Rush Attempts,Touchdowns,20+ Yard Runs,40+ Yard Runs,Longest Run,Fumbles
1,Ezekiel Elliott,1434,304,6,11,1,41,6
2,Saquon Barkley,1307,261,11,16,7,78,0
3,Todd Gurley,1251,256,17,11,0,36,0
4,Joe Mixon,1168,237,8,11,3,51,0
5,Chris Carson,1151,247,9,8,1,61,3
6,Christian McCaffrey,1098,219,7,6,3,59,2
7,Derrick Henry,1059,215,12,4,2,99,0
8,Adrian Peterson,1042,251,7,5,3,90,2
9,Phillip Lindsay,1037,192,9,8,3,65,0
10,Nick Chubb,996,192,8,11,4,92,0


### Defensive Stats